<a href="https://colab.research.google.com/github/SkShoheb33/Information-Retreval/blob/main/informationalRetreval1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
class InfoRetrival:
  def __init__(self,corpus):  
    self.corpus = corpus
    self.documents = []
    self.titles = []
    self.tokens = []
    self.termDocs = {}
    self.invertIndex = {}
    self.boolIndex = {}
    self.positionIndex = {}
    for i in os.listdir(corpus):
      self.documents.append(open(corpus+i).read())
      self.titles.append(i[:-4])





  def makeTerms(self,document):
    terms = document.split(" ")
    n = len(terms)
    for i in range(n):
      if not terms[i].islower():
        terms[i] = terms[i].lower()
      terms[i] = re.sub(r'[^a-zA-Z0-9]', '', terms[i])
    return terms



  def removeStopWords(self,terms):
    stopWords = ["i","am","it","is","the","we","was","a","an","and","or","to","so",".",";","!","@","#","$","%","^","&","*","(",")","{","}","[","]",":","'","/","<",">",'"',"+"]
    i = 0
    while i < len(terms):
      if terms[i] in stopWords  or len(terms[i])==0:
        terms.remove(terms[i])
      else:
        i+=1
    return terms



  def termsDocuments(self):
    for title,docs in zip(self.titles,self.documents):
      self.termDocs[title] = self.removeStopWords(self.makeTerms(docs))




  def stemming(self):
    ps = PorterStemmer()
    for title in self.termDocs:
      n = len(self.termDocs[title])
      for i in range(n):
        self.termDocs[title][i] = ps.stem(self.termDocs[title][i])



  def tokenize(self):
    self.termsDocuments()
    self.stemming()
    for title in self.termDocs:
      n = len(self.termDocs[title])
      for i in range(n):
        if self.termDocs[title][i] in self.tokens:
          continue
        self.tokens.append(self.termDocs[title][i])
    self.tokens.sort()




  def invertedIndex(self):
    self.tokenize()
    for token in self.tokens:
      for doc in self.termDocs:
        if token in self.termDocs[doc]:
          if token not in self.invertIndex:
            self.invertIndex[token] = []
          self.invertIndex[token].append(doc)




  def booleanIndex(self):
    self.tokenize()
    for i in self.tokens:
      for j in self.titles:
        if i not in self.boolIndex:
          self.boolIndex[i] = []
        if j in self.invertIndex[i]:
          self.boolIndex[i].append(1)
        else:
          self.boolIndex[i].append(0)


          
  def positionalIndex(self):
    ps = PorterStemmer()
    for i in self.tokens:
      self.positionIndex[i] = {}
      for j in self.titles:
        document = open(self.corpus+(j+".txt")).read().lower().split(" ")
        n = len(document)
        row = []
        for k in range(n):
          if i == ps.stem(document[k]):
            row.append(k)
        if len(row)>0:
          self.positionIndex[i][j] = row


  def printBooleanIndex(self):
    return pd.DataFrame(self.boolIndex,index=self.titles)




  def boolean_and(self,row1,row2):
    n = len(row1)
    for i in range(n):
      if row1[i] == row2[i] == 1:
        row1[i] = 1
      else:
        row1[i] = 0
    return row1



  def boolean_or(self,row1,row2):
    n = len(row1)
    for i in range(n):
      if row1[i] == 1 or row2[i] == 1:
        row1[i] = 1
      else:
        row1[i] = 0
    return row1



  def boolean_not(self,row):
    n = len(row)
    for i in range(n):
      if row[i] :
        row[i] = 0
      else:
        row[i] = 1
    return row



  def booleanQuery(self,query):
    query = query.split(" ")
    opearator = ""
    flag = False
    n = 10
    result = []
    for i in query:
      if i in ['and','or']:
        opearator = i
      elif i == 'not':
        flag = True
      else:
        if flag :
          flag = False
          result = self.boolean_not(self.boolIndex[i])
        if opearator == "and":
          result = self.boolean_and(result,self.boolIndex[i])
          operator = ""
        elif opearator == "or":
          result = self.boolean_or(result,self.boolIndex[i])
          operator = ""
        else:
          result = self.boolIndex[i]
    ans = []
    for i in range(n):
      if result[i]:
        ans.append(self.titles[i])
    return ans



  def phraseQuery(self,query):
    positions = {}
    query = query.split(" ")
    for word in query:
      positions[word] = self.positionIndex[word]
    # for 

In [ ]:
obj = InfoRetrival("/content/drive/MyDrive/stories/")
obj.invertedIndex()
obj.invertIndex
obj.booleanIndex()
obj.boolIndex
obj.printBooleanIndex()
# obj.booleanQuery("tiger or tiger")
obj.positionalIndex()
obj.positionIndex

In [ ]:

obj.positionIndex['onc']

{'doc3': [0],
 'doc7': [0],
 'doc8': [2, 1133, 1247],
 'doc9': [85],
 'doc10': [0],
 'doc5': [2]}

In [ ]:
obj.positionIndex['upon']

{'doc3': [1], 'doc7': [1], 'doc8': [3], 'doc10': [1], 'doc6': [45, 136]}